In [ ]:
require 'parser/current'
# opt-in to most recent AST format:
Parser::Builders::Default.emit_lambda              = true

In [ ]:
ruby_ast.type?

In [ ]:
require 'rubocop-ast'

In [ ]:
class Rule < Parser::AST::Processor
  include RuboCop::AST::Traversal

  def initialize(rewriter)
    @rewriter = rewriter
  end

  def create_range(begin_pos, end_pos)
    Parser::Source::Range.new(@rewriter.source_buffer, begin_pos, end_pos)
  end
end


class ReplaceRequire < Rule
  def on_send(node)
    if node.method_name == :require
      raise if node.child_nodes.size != 1
      req = node.child_nodes[0]
      raise if !req.str_type?
      @rewriter.replace(node.loc.expression, "import #{req.value}")
    elsif node.method_name == :require_relative
      raise if node.child_nodes.size != 1
      req = node.child_nodes[0]
      raise if !req.str_type?
      if req.value == "lib/baseline_model"
        @rewriter.replace(node.loc.expression, "from lib.baseline_model import BaselineModel")
      else
        raise if req.value != "lib/baseline_model"
      end
    end
  end
end

class ReplaceNew < Rule
  def on_send(node)
    if node.method_name == :new
      range = create_range(node.loc.dot.begin_pos, node.loc.selector.end_pos)
      @rewriter.replace(range, '')
      puts node.receiver
    end
  end
end

class AddParenthesesToMethods < Rule
  def on_send(node)
    if node.arguments.empty? && !(node.loc.expression.source_line.end_with?('()'))
      range = create_range(node.loc.expression.end_pos, node.loc.expression.end_pos)
      @rewriter.replace(range, '()')
      puts node.receiver
    end
  end
end

class ProcessHash < Rule
  def on_hash(node)
    # @rewriter.replace(node.loc.expression, node.loc.expression.source.gsub('=>',':'))
    
    args = []
    node.child_nodes.each do |pair_node|
      raise if !pair_node.pair_type?
      raise if pair_node.child_nodes.size != 2
      raise if !pair_node.child_nodes[0].str_type?
      args << "#{pair_node.child_nodes[0].value}=#{pair_node.child_nodes[1].loc.expression.source}"
    end
    @rewriter.replace(node.loc.expression, args.join(', '))
  end
end

In [ ]:
# Main method.
def process_file(file, rule_classes=[AddParenthesesToMethods, ReplaceRequire, ReplaceNew, ProcessHash])
  return unless File.exist?(file)
  code = File.read(file)
  frozen_lit = "# frozen_string_literal: true\n\n"
  if code.start_with?(frozen_lit)
    code = code[frozen_lit.size..]
  end
  
  rule_classes.each do |rule_class|
    code = process_rule(rule_class, code)
  end
  code
end

def process_rule(rule_class, code)
  source = RuboCop::AST::ProcessedSource.new(code, 2.7)
  source_buffer = source.buffer
  rewriter = Parser::Source::TreeRewriter.new(source_buffer)
  rule = rule_class.new(rewriter)
  source.ast.each_node { |n| rule.process(n) }
  rewriter.process
end

In [ ]:
file_name = 'model/simulationtests/humidity_control.rb'
file_name = 'model/simulationtests/test_port.rb'
file_name = 'model/simulationtests/baseline_sys01.rb'
file_name = 'model/simulationtests/baseline_sys02.rb'

In [ ]:
def port_file(file_name)
  code = process_file(file_name)
  code.gsub!('OpenStudio::Model::', 'openstudio.model.')
  code.gsub!('Dir.pwd()', 'None')
  puts "=" * 80
  puts code

  target_file = File.join(File.dirname(file_name), File.basename(file_name, '.rb') + '.py')
  puts "tatget file: #{target_file}"

  File.write(target_file, code)
end

In [ ]:
["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"].each do |sys_num|
  port_file("model/simulationtests/baseline_sys#{sys_num}.rb")
end

## Test bed

In [ ]:


class MyRule < Parser::AST::Processor
  include RuboCop::AST::Traversal

  def on_send(node)
    # puts "I found a on_send! #{node}"
    receiver_node, method_name, *arg_nodes = *node

    receiver_node = process(receiver_node) if receiver_node
    node.updated(nil, [
      receiver_node, method_name, *process_all(arg_nodes)
    ])
    if method_name == :require
      raise if arg_nodes.size != 1
      req = arg_nodes[0]
      raise if !req.str_type?
      puts req.value
    elsif method_name == :require_relative
      raise if arg_nodes.size != 1
      req = arg_nodes[0]
      raise if !req.str_type?
      raise if req.value != "lib/baseline_model"
      puts req.value
    end
  end
end

rule = MyRule.new
source.ast.each_node { |n| rule.process(n) }

In [ ]:
source_code = File.read(file_name)
source = RuboCop::AST::ProcessedSource.from_file(file_name, 2.7)

In [ ]:
rule.on_hash

In [ ]:
# code = "dehumidify_sch = OpenStudio::Model::ScheduleConstant.new(model)"
# source = RuboCop::AST::ProcessedSource.new(code, 2.7)

In [ ]:
node = source.ast.each_node.to_a[6]
node

In [ ]:
source.ast.each_node.to_a[68]

In [ ]:
$LIB_METHOD_NAMES = ['add_geometry',
 'add_windows',
 'set_constructions',
 'add_daylighting',
 'add_hvac',
 'set_space_types',
 'add_thermostats',
 'add_design_days',
 'force_year_description',
 'save_openstudio_osm',
 'add_standards',
 'add_schedule',
 'add_water_heater',
 'add_swh_loop',
 'add_swh_end_uses',
 'rename_loop_nodes',
 'rename_air_nodes'
]

In [ ]:
node.send_type?

In [ ]:
node.method_name

In [ ]:
source_code[node.loc.selector.begin_pos...node.loc.selector.end_pos]

In [ ]:
source_code

In [ ]:
node = source.ast.each_node.to_a[8]
node

In [ ]:
node.method_name

In [ ]:
node = source.ast.each_node.select { |node|
  next if !node.send_type?
  next if ! $LIB_METHOD_NAMES.include?(node.method_name.to_s)
  node
}[1]

In [ ]:
hash_node = node.arguments[0]
hash_node

In [ ]:
args = []
hash_node.child_nodes.each do |pair_node|
  raise if !pair_node.pair_type?
  raise if pair_node.child_nodes.size != 2
  raise if !pair_node.child_nodes[0].str_type?
  args << "#{pair_node.child_nodes[0].value}=#{pair_node.child_nodes[1].loc.expression.source}"
end
puts args.join(', ')

In [ ]:
pair_node = hash_node.child_nodes[0]
pair_node

In [ ]:
pair_node.child_nodes.size

In [ ]:
str_node = pair_node.child_nodes[0]

In [ ]:
str_node.value

In [ ]:
"#{pair_node.child_nodes[0].loc.expression.source}=#{pair_node.child_nodes[1].loc.expression.source}"

In [ ]:
child_node.child_nodes[0].value

In [ ]:
child_node.child_nodes[1].value

In [ ]:
str_node = child_node.child_nodes[0]
str_node

In [ ]:
node.loc.expression.source.gsub('=>',':')

In [ ]:
print(source_code[node.loc.expression.begin_pos...node.loc.expression.end_pos])
node.loc.expression.source.gsub('=>',':')

In [ ]:
node.loc.expression.begin_pos

In [ ]:
node = source.ast.each_node.to_a[24]
node

In [ ]:
node.if?

In [ ]:
n = node.child_nodes[0]
n

In [ ]:
n.method_name

In [ ]:
source_code[113...116]

In [ ]:
s = node.loc.expression.source_line
puts s
s.end_with?'()'

In [ ]:
source_code[node.loc.dot.begin_pos...node.loc.expression.end_pos]

In [ ]:
node.loc.expression.end

In [ ]:
source.ast.each_node.to_a[8]

In [ ]:
node.loc

In [ ]:
node.loc.expression.source_line.end_with?'()'

In [ ]:
node.arguments.empty?

In [ ]:
"BaselineModel.new".size

In [ ]:
node.receiver

In [ ]:
source_code[113...116]

In [ ]:
node.child_nodes

In [ ]:
node = source.ast.each_node.to_a[76]
node

In [ ]:
node = source.ast.each_node.to_a[77]
node

In [ ]:
node = source.ast.each_node.to_a[98]

In [ ]:
s = node.loc.expression.source_line
puts s
s.end_with?'()'

In [ ]:
node.arguments

In [ ]:
node.method_name

In [ ]:
node.sibling_index

In [ ]:
x = node.receiver
x

In [ ]:
x.const_type?

In [ ]:
x.module_name?

In [ ]:
x.class_name?

In [ ]:
x

In [ ]:
x.const_type?

In [ ]:
x.children[0].children[0]

In [ ]:
x.children[0]

In [ ]:
node.child_nodes

In [ ]:
x.const_name

In [ ]:
receiver_node, method_name, *arg_nodes = *node

In [ ]:
node.method_name

In [ ]:
node.child_nodes

In [ ]:
req = arg_nodes[0]

In [ ]:
req.str_type?

In [ ]:
req.value .

In [ ]:
node.loc.selector

In [ ]:
"require 'openstudio'".size

In [ ]:
"require".size

In [ ]:
node.loc.selector

In [ ]:
node.loc.expression.begin_pos

In [ ]:
node.loc.expression.end_pos

In [ ]:
node.loc